#### Глава 5. Работа с векторами слов
Узнаю - применение алгоритмов машинного обучения для генерации векторов слов, метод подобия (определяет близость смыслов объектов- контейнеров путем сравнения соответствующих векторов слов). Как повысить эффективность операций, в частности выбор ключевых слов, с помощью предварительной обработки?


In [1]:
import ru_core_news_lg
import spacy as sp
import sys

from spacy.symbols import ORTH, LEMMA
from IPython.display import Image
from spacy.tokens.doc import Doc
from spacy.vocab import Vocab

from spacy import displacy
from IPython.display import Image

nlp = sp.load('ru_core_news_lg')

### Метод similarity
Результат близкий к 1 показывает, что фразы или слова очень близки (диапазон значений степени подобия от 0 до 1)

In [2]:
doc = nlp(u"Я хочу спать в кроватке.")
doc2 = nlp(u"Я хочу спать на диване.")

# покажем близость фраз "Я хочу спать в кроватке" и "спать в кроватке"
print(doc.similarity(doc))

print(doc, " | ", doc[2:5], ":",  doc.similarity(doc[2:5]))
print(doc[3:5], " | ", doc2[3:5], ":",  doc[3:5].similarity(doc2[3:5]))

1.0
Я хочу спать в кроватке.  |  спать в кроватке : 0.9451089102124649
в кроватке  |  на диване : 0.4528334140777588


##### Cравниваем объект Token "кроватке" с объектом Span, содержащим один токен "диване"

In [3]:
token = doc[4:5][0]
print(token.similarity(doc2[4:5]))

0.4176931083202362


#### Использование сторонних пакетов векторов слов
Возникли проблемки с заргрузкой пакетика

#### Сравнение объектов spaCy. Применение семантического подобия для задач категоризации

In [4]:
doc = nlp(u"Я хочу купить эту прекрасную книгу в конце недели. Продажи апельсинов выросли за последний год. Как много вы знаете об этом типе деревьев?")
token = nlp(u'фрукты')[0]

for sent in doc.sents:
    print(sent.text)
    print('similarity to', token.text, 'is', token.similarity(sent),'\n')


Я хочу купить эту прекрасную книгу в конце недели.
similarity to фрукты is 0.18718963861465454 

Продажи апельсинов выросли за последний год.
similarity to фрукты is 0.26642706990242004 

Как много вы знаете об этом типе деревьев?
similarity to фрукты is 0.15825019776821136 



#### Выделение существительных как шаг предварительной обработки

In [5]:
simil = {}

for i, sent in enumerate(doc.sents):
    noun_span_list = [sent[j].text for j in range(len(sent)) if sent[j].pos_ == 'NOUN']
    print(noun_span_list)
    noun_span_str = ' '.join(noun_span_list)
    noun_span_doc = nlp(noun_span_str)
    simil.update({i:token.similarity(noun_span_doc)})

print(simil)


['книгу', 'конце', 'недели']
['Продажи', 'апельсинов', 'год']
['типе', 'деревьев']
{0: 0.11895324898625022, 1: 0.3911665157072323, 2: 0.11317716145060874}


In [6]:
Image(url="images/picture_8.png", width=500, height=300)

In [7]:
max_similarity = {}
dist = []

for i, sent in enumerate(doc.sents):
    noun_span_list = [sent[j].text for j in range(len(sent)) if sent[j].pos_ == 'NOUN']
    print(noun_span_list)
    
    dist = []
    for j, chunk in enumerate(noun_span_list):
        dist.append(abs((token.similarity(nlp(chunk)))))
    print(dist)

    max_similarity.update({i:max(dist)})

print(max_similarity)

['книгу', 'конце', 'недели']
[0.06992303473157861, 0.0128732583159894, 0.16338935322537068]
['Продажи', 'апельсинов', 'год']
[0.0, 0.4235835941789032, 0.11856940699250682]
['типе', 'деревьев']
[0.07038050812929242, 0.20813423873229375]
{0: 0.16338935322537068, 1: 0.4235835941789032, 2: 0.20813423873229375}


/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_72179/1997239711.py:10: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  dist.append(abs((token.similarity(nlp(chunk)))))


#### Выделение и сравнение именованных сущностей
метод отлично работает для английского языка, но для русского - ясно понятно

In [8]:
doc1 = nlp(u"Волгоград очень красивый город и если вы хотите его увидеть, то вам обязательно стоит посетить его. Это город который не оставит равнодушным никого, а так же здесь можно встретить очень много интересных людей.")
doc2 = nlp(u"Москва выглядит чудесно. Я не могу сказать, что это такое волшебное место, но точно, что оно очень красивое. Мне нравится здесь. И я хочу остаться здесь в Москве надолго.")
doc3 = nlp(u"Река Волга самая длинная в Европе. Длина реки составляет 3550 километров. В среднем течении она образует самую большую дельту в мире.")
docs = [doc1, doc2, doc3]

In [9]:
spans = {}
for j,doc in enumerate(docs):
    named_entity_span = [doc[i].text for i in range(len(doc)) if doc[i].ent_type != 0]
    print(named_entity_span)
    named_entity_span = ' '.join(named_entity_span)
    named_entity_span = nlp(named_entity_span)
    spans.update({j:named_entity_span})

['Волгоград']
['Москва', 'Москве']
['Европе']


In [10]:
print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))

token1 = nlp(u"Настя")
token2 = nlp(u"Анастасия")
print(token1.similarity(token2))

doc1 is similar to doc2: 0.0
doc1 is similar to doc3: 0.0
doc2 is similar to doc3: 0.0


/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_72179/653064147.py:1: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc1 is similar to doc2:',spans[0].similarity(spans[1]))
/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_72179/653064147.py:2: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc1 is similar to doc3:',spans[0].similarity(spans[2]))
/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_72179/653064147.py:3: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print('doc2 is similar to doc3:',spans[1].similarity(spans[2]))


0.0


/var/folders/c9/8zxgwvj510zc_3b1s0jbh5180000gn/T/ipykernel_72179/653064147.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  print(token1.similarity(token2))
